In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

input_dims = (1,120,1,)
l1_output_dim = 84
l2_output_dim = 10

t_max = 257

np.random.seed(seed=48)
weights1 = (np.random.rand(l1_output_dim, np.product(input_dims)) - 0.5) / 2
biases1 = (np.random.rand(l1_output_dim) - 0.5) / 2
weights2 = (np.random.rand(l2_output_dim, l1_output_dim) - 0.5) / 2
biases2 = (np.random.rand(l2_output_dim) - 0.5) / 2

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.Dense(weights=weights1, biases=biases1),
    layers.Dense(weights=weights2, biases=biases2),
    layers.MonitorLayer(),
])

hidden7 = quartz.probe(loihi_model.layers[2].blocks[18])
calc7 = quartz.probe(loihi_model.layers[2].blocks[18].neurons[0])

values = np.random.rand(np.product(input_dims)) / 2
inputs = quartz.utils.decode_values_into_spike_input(values, t_max)
quantized_values = (values*t_max).round()/t_max
# quantized_weights1 = (weight_acc*weights1).round()/weight_acc
# quantized_weights2 = (weight_acc*weights2).round()/weight_acc
# quantized_biases1 = (biases1*t_max).round()/t_max
# quantized_biases2 = (biases2*t_max).round()/t_max

model = nn.Sequential(
    nn.Linear(in_features=np.product(input_dims), out_features=l1_output_dim), nn.ReLU(),
    nn.Linear(in_features=l1_output_dim, out_features=l2_output_dim), nn.ReLU(),
)
model[0].weight = nn.Parameter(torch.tensor(weights1))
model[0].bias = nn.Parameter(torch.tensor(biases1))
model[2].weight = nn.Parameter(torch.tensor(weights2))
model[2].bias = nn.Parameter(torch.tensor(biases2))
model_output = model(torch.tensor(quantized_values)).detach().numpy()
model_output

In [ ]:
loihi_model(inputs, t_max)

In [ ]:
loihi_model(inputs, t_max)

In [ ]:
loihi_model(inputs, t_max)

In [ ]:
loihi_model.layers[0].vth_mant

In [ ]:
hidden7.output()

In [ ]:
calc7.plot()

In [ ]:
hidden7.plot()

## Conv Maxpool layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

input_dims = ( 1,2,2)
weight_dims = ( 3,1,2,2)

# (( 1,32,32), ( 6,1,5,5), 251),
# (( 6,14,14), (8,6,5,5), 251),
t_max = 2**9
weight_acc = 128
conv_kernel_size = weight_dims[2:]
pooling_kernel_size = [2,2]
pooling_stride = 2

np.random.seed(seed=47)
weights = (np.random.rand(*weight_dims)-0.5) / 4
biases = (np.random.rand(weight_dims[0])-0.5) / 2

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.Conv2D(weights=weights, biases=biases),
    layers.MaxPool2D(kernel_size=pooling_kernel_size, stride=pooling_stride),
    layers.MonitorLayer(),
])

trigg0 = quartz.probe(loihi_model.layers[1].blocks[0])
conv00 = quartz.probe(loihi_model.layers[1].blocks[1].neurons[0])
conv01 = quartz.probe(loihi_model.layers[1].blocks[1].neurons[1])
conv20 = quartz.probe(loihi_model.layers[1].blocks[7].neurons[0])
conv21 = quartz.probe(loihi_model.layers[1].blocks[7].neurons[1])
pool0 = quartz.probe(loihi_model.layers[2].blocks[0])
pool1 = quartz.probe(loihi_model.layers[2].blocks[0])

values = np.random.rand(np.product(input_dims))
inputs = quartz.utils.decode_values_into_spike_input(values, t_max)
quantized_values = (values*t_max).round()/t_max
quantized_weights = (weight_acc*weights).round()/weight_acc
quantized_biases = (biases*t_max).round()/t_max

model = nn.Sequential(
    nn.Conv2d(in_channels=weight_dims[1], out_channels=weight_dims[0], kernel_size=conv_kernel_size), nn.ReLU(),
    nn.MaxPool2d(kernel_size=pooling_kernel_size, stride=pooling_stride), nn.ReLU(),
)
model[0].weight = nn.Parameter(torch.tensor(quantized_weights))
model[0].bias = nn.Parameter(torch.tensor(quantized_biases))
model_output = model(torch.tensor(quantized_values.reshape(1, *input_dims[:3]))).squeeze().detach().numpy()
model_output.flatten()

In [ ]:
loihi_model(inputs, t_max)

In [ ]:
biases

In [ ]:
trigg0.plot()

In [ ]:
conv20.plot()

In [ ]:
conv21.plot()

## double maxconv

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

input_dims = ( 1,10,10)
weight_dims = ( 6,1,3,3)
weight_dims2 = (5,6,2,2)

t_max = 2**8
weight_acc = 255
conv_kernel_size = weight_dims[2:]
pooling_kernel_size = [2,2]
pooling_stride = 2

np.random.seed(seed=47)
np.set_printoptions(suppress=True)
weights = (np.random.rand(*weight_dims)-0.5) / 4 # np.zeros(weight_dims) #
weights2 = (np.random.rand(*weight_dims2)-0.5) / 4 # np.zeros(weight_dims) #
biases = (np.random.rand(weight_dims[0])-0.5)
biases2 = (np.random.rand(weight_dims2[0])-0.5)
#biases2[0] = 0.6
#biases2[1] = 0.5
biases = np.zeros(weight_dims[0])
biases2 = np.zeros(weight_dims2[0])

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.ConvPool2D(weights=weights, biases=biases, pool_kernel_size=pooling_kernel_size),
    layers.ConvPool2D(weights=weights2, biases=biases2, pool_kernel_size=pooling_kernel_size),
    layers.MonitorLayer(),
])
loihi_model1 = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.Conv2D(weights=weights, biases=biases),
    layers.MaxPool2D(kernel_size=pooling_kernel_size, stride=pooling_stride),
    layers.Conv2D(weights=weights2, biases=biases2),
    layers.MaxPool2D(kernel_size=pooling_kernel_size, stride=pooling_stride),
    layers.MonitorLayer(),
])

values = np.random.rand(np.product(input_dims))
inputs = quartz.utils.decode_values_into_spike_input(values, t_max)
quantized_values = (values*t_max).round()/t_max
quantized_weights = (weight_acc*weights).round()/weight_acc
quantized_weights2 = (weight_acc*weights2).round()/weight_acc
quantized_biases = (biases*t_max).round()/t_max

model = nn.Sequential(
    nn.Conv2d(in_channels=weight_dims[1], out_channels=weight_dims[0], kernel_size=conv_kernel_size), nn.ReLU(),
    nn.MaxPool2d(kernel_size=pooling_kernel_size, stride=pooling_stride), nn.ReLU(),
    nn.Conv2d(in_channels=weight_dims2[1], out_channels=weight_dims2[0], kernel_size=conv_kernel_size), nn.ReLU(),
    nn.MaxPool2d(kernel_size=pooling_kernel_size, stride=pooling_stride), nn.ReLU(),
)
model[0].weight = nn.Parameter(torch.tensor(weights))
model[0].bias = nn.Parameter(torch.tensor(biases))
model[4].weight = nn.Parameter(torch.tensor(weights2))
model[4].bias = nn.Parameter(torch.tensor(biases2))
model_output = model(torch.tensor(quantized_values.reshape(1, *input_dims[:3]))).squeeze().detach().numpy()
model_output

In [ ]:
loihi_model(inputs, t_max).reshape(*model_output.shape)

In [ ]:
loihi_model1(inputs, t_max).reshape(*model_output.shape)

In [ ]:
loihi_model.layers[1].blocks[-1].neurons[-1].incoming_synapses()

## convpool - conv

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

# input_dims = ( 5,14,14)
# weight_dims = ( 8,5,5,5)
# weight_dims2 = (120,8,5,5)
input_dims = ( 1,7,7)
weight_dims = ( 2,1,2,2)
weight_dims2 = (4,2,3,3)

# input_dims = ( 1,6,6)
# weight_dims = ( 2,1,3,3)
# weight_dims2 = (3,2,2,2)

t_max = 2**9
weight_acc = 2**7
conv_kernel_size = weight_dims[2:]
pooling_kernel_size = [2,2]
pooling_stride = 2

np.random.seed(seed=34)
np.set_printoptions(suppress=True)
weights = (np.random.rand(*weight_dims)-0.5) / 4 # np.zeros(weight_dims) #
weights2 = (np.random.rand(*weight_dims2)-0.5) / 4 # np.zeros(weight_dims) #
biases = (np.random.rand(weight_dims[0])-0.5) / 2
biases2 = (np.random.rand(weight_dims2[0])-0.5) / 2

biases = np.zeros(weight_dims[0]) # (np.random.rand(weight_dims[0])-0.5) / 2
biases2 = np.zeros(weight_dims2[0])

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.ConvPool2D(weights=weights, biases=None, pool_kernel_size=pooling_kernel_size, pool_stride=pooling_stride, weight_acc=weight_acc),
    layers.Conv2D(weights=weights2, biases=None, weight_acc=weight_acc),
    layers.MonitorLayer(),
])
loihi_model1 = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.Conv2D(weights=weights, biases=None, weight_acc=weight_acc),
    layers.MaxPool2D(kernel_size=pooling_kernel_size, stride=pooling_stride),
    layers.Conv2D(weights=weights2, biases=None, weight_acc=weight_acc),
    layers.MonitorLayer(),
])

trigger0 = quartz.probe(loihi_model.layers[0].blocks[-1])
trigger1 = quartz.probe(loihi_model.layers[1].blocks[0])
bias0 = quartz.probe(loihi_model.layers[2].blocks[0])
#bias1 = quartz.probe(loihi_model.layers[2].blocks[3])
relco0 = quartz.probe(loihi_model.layers[2].blocks[2])
#relco1 = quartz.probe(loihi_model.layers[2].blocks[5])
calc0 = quartz.probe(loihi_model.layers[2].blocks[2].neurons[0])
calc1 = quartz.probe(loihi_model.layers[2].blocks[2].neurons[1])

probes = [quartz.probe(block) for block in loihi_model.layers[1].output_blocks()]

# relco_alt0 = quartz.probe(loihi_model1.layers[3].output_blocks()[0])
# relco_alt1 = quartz.probe(loihi_model1.layers[3].output_blocks()[1])

values = np.random.rand(np.product(input_dims)) / 2
inputs = quartz.utils.decode_values_into_spike_input(values, t_max)

model = nn.Sequential(
    nn.Conv2d(in_channels=weight_dims[1], out_channels=weight_dims[0], kernel_size=conv_kernel_size), nn.ReLU(),
    nn.MaxPool2d(kernel_size=pooling_kernel_size, stride=pooling_stride), nn.ReLU(),
    nn.Conv2d(in_channels=weight_dims2[1], out_channels=weight_dims2[0], kernel_size=conv_kernel_size), nn.ReLU(),
)
model[0].weight = nn.Parameter(torch.tensor(weights))
model[0].bias = nn.Parameter(torch.tensor((biases)))
model[4].weight = nn.Parameter(torch.tensor(weights2))
model[4].bias = nn.Parameter(torch.tensor((biases2)))
model_output = model(torch.tensor(values.reshape(1, *input_dims[:3]))).squeeze().detach().numpy()
model_output

In [ ]:
loihi_model(inputs, t_max).reshape(model_output.shape)

In [ ]:
loihi_model1(inputs, t_max).reshape(model_output.shape)

In [ ]:
ok = [probe.plot() for probe in probes]

In [ ]:
[probe.output()[1] for probe in probes]

In [ ]:
probe = probes[-3]

In [ ]:
probe.plot()

In [ ]:
probes[-3].output()

In [ ]:
probe.loihi_probe

In [ ]:
convpool1.plot()

In [ ]:
convpool0.plot()

In [ ]:
convpool10.plot()

In [ ]:
convpool11.plot()

In [ ]:
convpool2.plot()

In [ ]:
convpool3.plot()

In [ ]:
convpool4.plot()

In [ ]:
loihi_model.layers[2].blocks

In [ ]:
calc0.plot()

In [ ]:
loihi_model.layers[1].output_blocks()